# Homework 1 Problem 1

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#Load data

data = pd.read_csv("./data/resistivity_table.csv", delimiter = "\t", header = 0)

In [3]:
data

,element,density,resistivity_77,resistivity_273,lattice_constant
0,Ag,5.860000e+28,3.000000e-09,1.510000e-08,4.078200e-10
1,Au,5.900000e+28,5.000000e-09,2.040000e-08,4.085300e-10
2,Al,1.810000e+29,3.000000e-09,2.450000e-08,4.049500e-10


In [4]:
#Class for storing data

class Element:
    
    def __init__(self, name, density, lattice_constant, resistivity_list, temperature_list):
        self.name = name
        self.density = density
        self.lattice_constant = lattice_constant
        self.resistivity = {}
        if len(resistivity_list) == len(temperature_list):
            for i in range(len(resistivity_list)):
                self.resistivity[temperature_list[i]] = resistivity_list[i]

In [5]:
#Units

density_units = 10 ** 28 #[m^-3]
lattice_units = 10 ** (-12) # [m]
resistivity_units = 10 ** (-8) # [ohm * m]

In [6]:
temperatures = [77, 273]

ag = Element('Ag', data['density'][0], data['lattice_constant'][0], 
             [data['resistivity_77'][0], data['resistivity_273'][0]], temperatures)
au = Element('Au', data['density'][1], data['lattice_constant'][1],
             [data['resistivity_77'][1], data['resistivity_273'][1]], temperatures)
al = Element('Al', data['density'][2], data['lattice_constant'][2],
             [data['resistivity_77'][2], data['resistivity_273'][2]], temperatures)

In [7]:
ag.__dict__

{'name': 'Ag',
 'density': 5.86e+28,
 'lattice_constant': 4.0782e-10,
 'resistivity': {77: 3e-09, 273: 1.51e-08}}

In [8]:
elements = [ag, au, al]

## Basic Theory

Ohm's law:

\begin{equation}
\vec{j} = \sigma \vec{E}
\end{equation}

In Drude model, electrical conductivity is given as:

\begin{equation}
\sigma = \frac{ne^2\tau}{m}
\end{equation}

## Statement of problem

Using Drude model, calculate scattering time and mean free path of electrons.

### Formulas

Using electrical conductivity from Drude model, we can obtain expression for scattering time:

\begin{equation}
\tau = \frac{m}{\rho n e^2}
\end{equation}

Mean free path is given by:

\begin{equation}
l = v \tau
\end{equation}

where $v$ can be estimated using equation:

\begin{equation}
\overline{E_k} = \frac{1}{2}mv^2 = \frac{3}{2} kT
\end{equation}

So we get expression for mean free path:

\begin{equation}
l = \tau \sqrt{\frac{3kT}{m}}
\end{equation}

In [9]:
from scipy.constants import e, k, m_e

def tau(density, resistivity):
    return m_e / (e ** 2 * density * resistivity)

def mean_free_path(tau, temperature):
    return tau * np.sqrt(3 * k * temperature / m_e)

In [10]:
results_dict = {}

In [11]:
for element in elements:
    results_dict[element] = {}
    results_dict[element]['element'] = element.name
    for temperature in temperatures:
        results_dict[element]['tau_' + str(temperature)] = tau(element.density, element.resistivity[temperature])
        results_dict[element]['mfp_' + str(temperature)] = \
        mean_free_path(results_dict[element]['tau_' + str(temperature)], temperature)

In [12]:
results_dict

{<__main__.Element at 0x7f6343534e48>: {'element': 'Ag',
  'tau_77': 2.0185956722945247e-13,
  'mfp_77': 1.194407162831284e-08,
  'tau_273': 4.010454978068592e-14,
  'mfp_273': 4.468200651380432e-09},
 <__main__.Element at 0x7f6343534d68>: {'element': 'Au',
  'tau_77': 1.2029461667436522e-13,
  'mfp_77': 7.11785692290643e-09,
  'tau_273': 2.948397467508951e-14,
  'mfp_273': 3.284921924543443e-09},
 <__main__.Element at 0x7f6343534e80>: {'element': 'Al',
  'tau_77': 6.535342894831996e-14,
  'mfp_77': 3.866975676348797e-09,
  'tau_273': 8.002460687549382e-15,
  'mfp_273': 8.915846269884966e-10}}

In [13]:
results = pd.DataFrame(results_dict)

for i in range(len(elements)):
    results = results.rename(index = str, columns = {elements[i]: i})

In [14]:
results

,0,1,2
element,Ag,Au,Al
mfp_273,4.4682e-09,3.28492e-09,8.91585e-10
mfp_77,1.19441e-08,7.11786e-09,3.86698e-09
tau_273,4.01045e-14,2.9484e-14,8.00246e-15
tau_77,2.0186e-13,1.20295e-13,6.53534e-14
